In [1]:
from parcels.plotting import create_parcelsfig_axis
import xarray as xr
import numpy as np
import matplotlib.animation as animation
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def makemov(filename):
    pfile = xr.open_dataset(str(filename), decode_cf=True)
    lon = np.ma.filled(pfile.variables['lon'], np.nan)
    lat = np.ma.filled(pfile.variables['lat'], np.nan)
    z = np.ma.filled(pfile.variables['z'], np.nan)
    temp = np.ma.filled(pfile.variables['temp'], np.nan)
    time = np.ma.filled(pfile.variables['time'], np.nan)
    pfile.close()

    plt, fig, ax, cartopy = create_parcelsfig_axis(spherical=True)

    plottimes = np.unique(time)
    plottimes = plottimes[~np.isnat(plottimes)]
    b = (time == plottimes[0]) & (z > 50.)
    scat_sink = ax.scatter(lon[b], lat[b], s=20, color='k', transform=cartopy.crs.Geodetic())
    b = (time == plottimes[0]) & (z == 50.)
    scat_dwell = ax.scatter(lon[b], lat[b], c=temp[b], s=35, edgecolor='k', transform=cartopy.crs.Geodetic())
    ttl = ax.set_title('Particles at time ' + str(plottimes[0]))
    ax.set_xlim(np.nanmin(lon), np.nanmax(lon))
    ax.set_ylim(np.nanmin(lat), np.nanmax(lat))
    frames = np.arange(1, 350)

    def animate(t):
        b = (time == plottimes[t]) & (z > 50.)
        scat_sink.set_offsets(np.vstack((lon[b], lat[b])).transpose())
        b = (time == plottimes[t]) & (z == 50.)
        scat_dwell.set_offsets(np.vstack((lon[b], lat[b])).transpose())
        scat_dwell.set_array(temp[b])
        ttl.set_text('Particle at time ' + str(plottimes[t]))
        return scat_dwell,

    rc('animation', html='html5')
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=False)
    plt.close()
    return anim

makemov("corefootprint_particles.nc")

/Users/erik/miniconda2/envs/py2_parcels_cartopy/lib/python2.7/site-packages/numpy/lib/arraysetops.py:285: FutureWarning: In the future, NAT != NAT will be True rather than False.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/Users/erik/miniconda2/envs/py2_parcels_cartopy/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  
/Users/erik/miniconda2/envs/py2_parcels_cartopy/lib/python2.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in greater
